In [4]:
from simpleai.search import CspProblem, backtrack, min_conflicts, hill_climbing, simulated_annealing, genetic
from simpleai.search import backtrack
from simpleai.search.csp import (min_conflicts)
from simpleai.search.local import hill_climbing, simulated_annealing, genetic
import random

# Tự định nghĩa heuristic variable: chọn biến có ít giá trị nhất (most constrained)
def most_constrained_variable(variables, domains, constraints, assigned):
    # Chọn biến chưa gán có domain nhỏ nhất
    unassigned = [v for v in variables if v not in assigned]
    return min(unassigned, key=lambda var: len(domains[var]))
# Tự định nghĩa heuristic value: chọn giá trị ít gây ràng buộc nhất (least constraining)
def least_constraining_value(variable, domains, constraints, assigned):
    # Đếm số biến bị ảnh hưởng nếu chọn giá trị này
    def count_conflicts(value):
        count = 0
        for var in domains:
            if var != variable and var not in assigned:
                for val in domains[var]:
                    if not constraints.get((min(variable,var), max(variable,var)), lambda v1,v2,v3,v4: True)(variable, value, var, val):
                        count += 1
        return count
    values = domains[variable]
    return sorted(values, key=count_conflicts)
  
# Định nghĩa bài toán 5 hậu dưới dạng CSP
def create_5_queens_csp():
    variables = [f"Q{i}" for i in range(5)]
    domains = {v: list(range(5)) for v in variables}

    constraints = []
    for i in range(5):
        for j in range(i + 1, 5):
            v1, v2 = f"Q{i}", f"Q{j}"

            # Hàm nhận đúng 2 giá trị, simpleai sẽ truyền trực tiếp
            def make_constraint(i=i, j=j):
                def constraint(val_i, val_j):
                    # khác hàng
                    if val_i == val_j:
                        return False
                    # khác đường chéo
                    return abs(i - j) != abs(val_i - val_j)
                return constraint

            constraints.append(((v1, v2), make_constraint()))

    return CspProblem(variables, domains, constraints)

# Hàm chạy backtracking với các tùy chọn heuristic và AC-3
def run_backtracking(ac3=False, var_heuristic=None, val_heuristic=None):
    problem = create_5_queens_csp()
    # Nếu muốn dùng AC-3, bật inference=True, simpleai sẽ tự xử lý
    result = backtrack(problem,
                       variable_heuristic=var_heuristic,
                       value_heuristic=val_heuristic,
                       inference=ac3)  # inference=True sẽ kích hoạt AC-3
    return result

# Hàm chạy local search (GA, SA, Hill Climbing)
def run_local_search(method='hill_climbing'):
    problem = create_5_queens_csp()

    # Khởi tạo trạng thái ngẫu nhiên
    initial_state = {var: random.choice(problem.domains[var]) for var in problem.variables}

    if method == 'hill_climbing':
        result = hill_climbing(problem, initial_state=initial_state)
    elif method == 'simulated_annealing':
        result = simulated_annealing(problem, initial_state=initial_state)
    elif method == 'genetic':
        result = genetic(problem, population_size=50, generations_limit=100)
    else:
        raise ValueError("Phương pháp không hợp lệ")

    return result.state

# Chạy thử nghiệm và in kết quả
if __name__ == '__main__':
    print("Backtracking không AC-3, variable_heuristic=most_constrained_variable, value_heuristic=least_constraining_value")
    sol1 = run_backtracking(ac3=False,
                           var_heuristic=most_constrained_variable,
                           val_heuristic=least_constraining_value)
    print(sol1)

    print("\nBacktracking có AC-3, variable_heuristic=most_constrained_variable, value_heuristic=least_constraining_value")
    sol2 = run_backtracking(ac3=True,
                           var_heuristic=most_constrained_variable,
                           val_heuristic=least_constraining_value)
    print(sol2)

    print("\nLocal search - Hill Climbing")
    sol3 = run_local_search('hill_climbing')
    print(sol3)

    print("\nLocal search - Simulated Annealing")
    sol4 = run_local_search('simulated_annealing')
    print(sol4)

    print("\nLocal search - Genetic Algorithm")
    sol5 = run_local_search('genetic')
    print(sol5)


Backtracking không AC-3, variable_heuristic=most_constrained_variable, value_heuristic=least_constraining_value


TypeError: unsupported operand type(s) for -: 'tuple' and 'tuple'